In [1]:
import ebooklib
from ebooklib import epub
import re
import os

In [2]:
def read_epub_book(epub_file, ID):
    book = epub.read_epub(epub_file)
    text = ""

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            content = re.sub("&#13;", ' ', content)
            text += content
            #paragraphs.extend(content.strip().split("&#13;"))
    book_item = {'text': text, 'bookID':ID}
    return book_item

In [3]:
read_epub_book('NewsFromNowhereWilliamMorris.epub',1)

d:\ARC\UCL\Term 01\RC 11\Skills\Text_Model\.venv\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


{'text': ' The Project Gutenberg eBook of News from Nowhere; Or, An Epoch of Rest This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: News from Nowhere; Or, An Epoch of Rest Author: William Morris Release date: June 1, 2002 [eBook #3261] Most recently updated: April 20, 2020 Language: English *** START OF THE PROJECT GUTENBERG EBOOK NEWS FROM NOWHERE; OR, AN EPOCH OF REST *** Transcribed from the 1908 Longmans, Green, and Co. edition by David Price, email ccx074@pglaf.org NEWS FROM NOWHERE or AN EPOCH OF REST being some chapters from A UTOPIAN ROMANCE by WILLIAM MORRIS, author of ‘the earthly pa

In [4]:
## Read epub into paragraphs of max length

In [5]:
def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

In [6]:
def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 100, 2500)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [7]:
read_epub_paragraphs('NewsFromNowhereWilliamMorris.epub',1)

[{'paragraph': ' You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.',
  'nr': 1,
  'bookID': 1},
 {'paragraph': 'gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.',
  'nr': 2,
  'bookID': 1},
 {'paragraph': ' Title: News from Nowhere; Or, An Epoch of Rest Author: William Morris Release date: June 1, 2002 [eBook #3261] Most recently updated: April 20, 2020 Language: English *** START OF THE PROJECT GUTENBERG EBOOK NEWS FROM NOWHERE; OR, AN EPOCH OF REST *** Transcribed from the 1908 Longmans, Green, and Co.',
  'nr': 3,
  'bookID': 1},
 {'paragraph': ' edition by David Price, email ccx074@pglaf.org NEWS FROM NOWHERE or AN EPOCH OF REST being some chapters from A UTOPIAN ROMANCE by WILLIAM MORRIS, author of ‘the earthly paradise.',
  'nr': 4,
  'bookID': 1},
 {'paragraph': '’ TENTH IMPRESSION LONGMANS, 

In [8]:
# Vectorise Text - 1

In [9]:
# first line needs to be set to where you have the c library of enchant
#os.environ['PYENCHANT_LIBRARY_PATH'] #= '/opt/homebrew/lib/libenchant-2.dylib'
import enchant

In [10]:
ENGLISH_DICT1 = enchant.request_dict("en_GB")
ENGLISH_DICT2 = enchant.request_dict("en_US")

In [11]:
def is_english_word(word):
    # Initialize the Enchant English dictionary
    return (ENGLISH_DICT1.check(word) or ENGLISH_DICT2.check(word))

In [12]:
is_english_word('test')

True

In [13]:
## Then we can start vectorising
### First step is to preprocess the paragraphs

In [14]:
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abdulmù\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abdulmù\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
paragraphs = read_epub_paragraphs('NewsFromNowhereWilliamMorris.epub',1)

d:\ARC\UCL\Term 01\RC 11\Skills\Text_Model\.venv\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [16]:
#Adjust to process the enite book
paragraph = paragraphs[25]['paragraph']


In [17]:
#Block to process everything
#for paragraph_info in paragraphs:
    #paragraph = paragraph_info['paragraph']

paragraph = paragraphs[36]['paragraph']

paragraph

' Of course when I had my head above water again I turned towards the tide, and my eyes naturally sought for the bridge, and so utterly astonished was I by what I saw, that I forgot to strike out, and went spluttering under water again, and when I came up made straight for the boat; for I felt that I must ask some questions of my waterman, so bewildering had been the half-sight I had seen from the face of the river with the water hardly out of my eyes; though by this time I was quit of the slumbrous and dizzy feeling, and was wide-awake and clear-headed.'

In [18]:
words = gensim.utils.simple_preprocess(paragraph, min_len = 3, deacc=True)
print(words)

['course', 'when', 'had', 'head', 'above', 'water', 'again', 'turned', 'towards', 'the', 'tide', 'and', 'eyes', 'naturally', 'sought', 'for', 'the', 'bridge', 'and', 'utterly', 'astonished', 'was', 'what', 'saw', 'that', 'forgot', 'strike', 'out', 'and', 'went', 'spluttering', 'under', 'water', 'again', 'and', 'when', 'came', 'made', 'straight', 'for', 'the', 'boat', 'for', 'felt', 'that', 'must', 'ask', 'some', 'questions', 'waterman', 'bewildering', 'had', 'been', 'the', 'half', 'sight', 'had', 'seen', 'from', 'the', 'face', 'the', 'river', 'with', 'the', 'water', 'hardly', 'out', 'eyes', 'though', 'this', 'time', 'was', 'quit', 'the', 'slumbrous', 'and', 'dizzy', 'feeling', 'and', 'was', 'wide', 'awake', 'and', 'clear', 'headed']


In [19]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
print(lemmatized_words)

['course', 'when', 'had', 'head', 'above', 'water', 'again', 'turned', 'towards', 'the', 'tide', 'and', 'eye', 'naturally', 'sought', 'for', 'the', 'bridge', 'and', 'utterly', 'astonished', 'wa', 'what', 'saw', 'that', 'forgot', 'strike', 'out', 'and', 'went', 'spluttering', 'under', 'water', 'again', 'and', 'when', 'came', 'made', 'straight', 'for', 'the', 'boat', 'for', 'felt', 'that', 'must', 'ask', 'some', 'question', 'waterman', 'bewildering', 'had', 'been', 'the', 'half', 'sight', 'had', 'seen', 'from', 'the', 'face', 'the', 'river', 'with', 'the', 'water', 'hardly', 'out', 'eye', 'though', 'this', 'time', 'wa', 'quit', 'the', 'slumbrous', 'and', 'dizzy', 'feeling', 'and', 'wa', 'wide', 'awake', 'and', 'clear', 'headed']


In [20]:
stop_words = stopwords.words("english")

In [21]:
filtered_words = [word for word in lemmatized_words if ((word not in stop_words)) and(is_english_word(word))]
print(filtered_words)

['course', 'head', 'water', 'turned', 'towards', 'tide', 'eye', 'naturally', 'sought', 'bridge', 'utterly', 'astonished', 'saw', 'forgot', 'strike', 'went', 'spluttering', 'water', 'came', 'made', 'straight', 'boat', 'felt', 'must', 'ask', 'question', 'waterman', 'bewildering', 'half', 'sight', 'seen', 'face', 'river', 'water', 'hardly', 'eye', 'though', 'time', 'quit', 'slumbrous', 'dizzy', 'feeling', 'wide', 'awake', 'clear', 'headed']


In [22]:
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print(stemmed_words)


['cours', 'head', 'water', 'turn', 'toward', 'tide', 'eye', 'natur', 'sought', 'bridg', 'utterli', 'astonish', 'saw', 'forgot', 'strike', 'went', 'splutter', 'water', 'came', 'made', 'straight', 'boat', 'felt', 'must', 'ask', 'question', 'waterman', 'bewild', 'half', 'sight', 'seen', 'face', 'river', 'water', 'hardli', 'eye', 'though', 'time', 'quit', 'slumbrou', 'dizzi', 'feel', 'wide', 'awak', 'clear', 'head']


In [23]:
processed_doc = " ".join(stemmed_words)
print(processed_doc)

cours head water turn toward tide eye natur sought bridg utterli astonish saw forgot strike went splutter water came made straight boat felt must ask question waterman bewild half sight seen face river water hardli eye though time quit slumbrou dizzi feel wide awak clear head


In [24]:
## _For homework:
#combine the above to a function that can process a list of paragraphs and return a list of processed paragraphs (hint: only assign the stop words, lemmatizer and stemmer once)_

In [25]:
def preprocess(paragraphs):
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    processed_paragraphs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word not in stop_words and is_english_word(word)]
        stemmed_words = [stemmer.stem(word) for word in filtered_words]
        processed_paragraph = " ".join(stemmed_words)
        processed_paragraphs.append(processed_paragraph)

    return processed_paragraphs

In [26]:
processedDoc = preprocess([p['paragraph'] for p in paragraphs]) #preprocess is the function you have to define

In [27]:
processedDoc

['may copi give away use term project licens includ onlin',
 'org locat unit state check law countri locat use',
 'titl news nowher epoch rest author morri releas date recent updat languag english start project news nowher epoch rest transcrib green',
 'edit price email org news nowher epoch rest chapter utopian romanc morri author earthli paradis',
 'tenth impress green paternost row new york right reserv first print serial commonw',
 'thenc reprint mass first english edit revis reef amp turner reprint march',
 'press edit sinc reprint march august',
 'chapter discuss bed leagu say friend one night brisk convers discuss would happen morrow revolut final shade vigor statement variou friend view futur fulli develop new societi',
 'say friend consid subject discuss good temper present use public meet lectur debat listen other opinion could scarc expect event alway attempt speak togeth custom peopl ordinari polit societi convers subject interest',
 'rest six person present consequ six sec

### Now turn these into vectors

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorizer = TfidfVectorizer(min_df=3)

In [30]:
tfidf_matrix = vectorizer.fit_transform(processedDoc)

In [31]:
tfidf_matrix

<1799x1928 sparse matrix of type '<class 'numpy.float64'>'
	with 30619 stored elements in Compressed Sparse Row format>

In [32]:
len(processedDoc)

1799

In [33]:
vectorizer.get_feature_names_out()

array(['abid', 'abl', 'abod', ..., 'younger', 'youngster', 'youth'],
      dtype=object)

In [34]:
len(vectorizer.get_feature_names_out())

1928

### Now we can search

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# Vectorise a new paragraph:
query = 'country'
processedQuery = preprocess([query])[0]
print(processedQuery)

countri


In [37]:
query_vector = vectorizer.transform([processedQuery])

In [38]:
query_vector

<1x1928 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [39]:
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [40]:
similarities

array([[0.        , 0.20045227, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [41]:
len(similarities[0])

1799

In [42]:
nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

In [43]:
nearest_neighbor_index, similarity_score

(1280, 0.5104595209957232)

In [44]:
paragraphs[1280]

{'paragraph': ' “Yes,” said I, much moved; “that and more. Often—in my country I have seen that wretched change you have spoken of, from the fresh handsome country lass to the poor draggle-tailed country woman.',
 'nr': 1281,
 'bookID': 1}

## This works quite well, but with a lot of data it becomes too heavy, so we apply SVD for dimensionality reduction

In [45]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [46]:
# Perform Truncated SVD for dimensionality reduction
n_components = 250  # Number of components to keep after reduction
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)

# The reduced_matrix now contains the dimensionality-reduced data

# If you want to project new data into the reduced space, you can use:
#new_data = ...  # Your new sparse matrix data
#reduced_new_data = svd.transform(new_data)

In [47]:
reduced_query_vec = svd.transform(query_vector)

In [48]:
len(reduced_query_vec[0])

250

In [49]:
similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)

In [50]:
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]

In [51]:
nearest_neighbor_index

1280

In [52]:
paragraphs[889]

{'paragraph': ' Some again were steps on the path of ‘State Socialism’; the most important of which can be speedily summed up.',
 'nr': 890,
 'bookID': 1}

# Vectorise Text - 2

In [53]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# https://radimrehurek.com/gensim/models/doc2vec.html

#### Define another preprocess function that return lemmatized (not stemmed) filtered words in a list, like this: